In [1]:
!nvidia-smi

Sun Jun 25 09:58:54 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   63C    P8    11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install -U git+https://github.com/huggingface/transformers@de9255de27abfcae4a1f816b904915f0b1e23cd9
!pip install torch --upgrade
!pip install datasets==2.10.1
!pip install faiss-gpu faiss-cpu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/transformers (to revision de9255de27abfcae4a1f816b904915f0b1e23cd9) to /tmp/pip-req-build-h06kc40i
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-h06kc40i
  Running command git rev-parse -q --verify 'sha^de9255de27abfcae4a1f816b904915f0b1e23cd9'
  Running command git fetch -q https://github.com/huggingface/transformers de9255de27abfcae4a1f816b904915f0b1e23cd9
  Running command git checkout -q de9255de27abfcae4a1f816b904915f0b1e23cd9
  Resolved https://github.com/huggingface/transformers to commit de9255de27abfcae4a1f816b904915f0b1e23cd9
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import torch
import numpy as np
import pandas as pd

from transformers import AutoTokenizer, AutoModel

import nltk
from datasets import load_dataset, Dataset
from collections import defaultdict
import json
from collections import Counter
from tqdm import tqdm
from random import sample

## Creating Dataset

In [4]:
# Create dataset
corpus = [
    "Дан двухатомный идеальный газ, молекулы которого обладают электрическим дипольным моментом d {|d| = d0 = const}. Газ находится в постоянном однородном электрическом поле e0 и имеет температуру Т. Требуется: а) найти добавок к свободной энергии рассматриваемого идеального газа, обусловленный взаимодействием дипольных моментов d с электрическим полем е0. Число молекул в газе N, объем газа V; б) зная результат пункта а), найти вклад в теплоемкость газа, обусловленный взаимодействием диполей d с полем e0; в) найти распределение по углам электрических дипольных моментов d в поле e0; г) зная результат пункта в), найти выражение для диэлектрической проницаемости газа в случае слабого поля e0.",
    "Пользуясь интегральным исчислением, решить задачи 7.12, 7.13 и 7.14 аналитически. 7.12. Пользуясь численными методами, найти положение центра масс полукруга. Для простоты расчета положить R==1,00. 7.13. Пользуясь численными методами, найти положение центра масс полушара. 7.14.Пользуясь численными методами, найти положение центра масс прямого кругового конуса высотой h=1, образующая которого составляет угол а с высотой.",
    "Заряд q распределен с постоянной поверхностной плотностью по поверхности полусферы радиуса а, лежащей в области z > 0 декартовой системы координат. Граничная окружность полусферы лежит в плоскости z = 0, центр этой окружности совпадает с началом координат. 1) Найти потенциал ф во всех точках плоскости z = 0, выразив его как функцию расстояния r до начала координат. 2) Как ориентирован вектор электрического поля в плоскости z = 0 при r < а? 3) Найти потенциал ф(z) на оси z. 4) Найти приближенно потенциал ф(r) на больших расстояниях от полусферы (|r| >> а) в присутствии дополнительного точечного заряда противоположного знака (-q), расположенного в начале координат.",
    "Материальная точка массой 0,01 кг совершает гармонические колебания с периодом 2 с. Полная энергия колеблющейся точки 10-4 Дж. Найти амплитуду колебаний, написать уравнение колебаний, найти наибольшее значение силы, действующей на точку."
          ]
df = pd.DataFrame({'text':corpus})
dataset = Dataset.from_pandas(df)

In [5]:
def cls_pooling(model_output):
    return model_output.last_hidden_state[:, 0]

In [6]:
def get_embeddings(text_list):
    encoded_input = tokenizer(
        text_list, padding=True, truncation=True, max_length = 200, return_tensors="pt"
    )
    encoded_input = {k: v for k, v in encoded_input.items()}
    model_output = model(**encoded_input)
    return cls_pooling(model_output)

# Compare models

In [7]:
import time

start = time.time()
print("hello")
end = time.time()
print(end - start)

hello
0.0017540454864501953


In [8]:
models = [{'Name': 'Sber SBERT 2021', 'Size': '1.71 GB', 'Path': "sberbank-ai/sbert_large_nlu_ru"},
          {'Name': 'DeepPavlov RUBERT 2021', 'Size': '711 MB', 'Path': "DeepPavlov/rubert-base-cased-sentence"},
          {'Name': 'RUBERT  tiny 2022', 'Size': '120 MB', 'Path': "cointegrated/rubert-tiny2"}
     ]

In [9]:
# DF to compare models
compare_models = []
queries = [
    "Пользуясь интегральным исчислением, решить задачи 7.12, 7.13 и 7.14 аналитически. 7.12. Пользуясь численными методами, найти положение центра масс полукруга. Для простоты расчета положить R==1,00. 7.13. Пользуясь численными методами, найти положение центра масс полушара. 7.14.Пользуясь численными методами, найти положение центра масс прямого кругового конуса высотой h=1, образующая которого составляет угол а с высотой.",
    "идеальный газ",
    "маятник"
           ]

#Loop on every model in list
for model_ in models:
  tokenizer = AutoTokenizer.from_pretrained(model_['Path'])
  model = AutoModel.from_pretrained(model_['Path'])

  # get embeddings for our text
  embeddings_dataset = dataset.map(lambda x: {"embeddings": get_embeddings(x["text"]).detach().numpy()[0]})
  # add faiss index to embeddings
  embeddings_dataset.add_faiss_index(column="embeddings")

  for query in queries:
    start = time.time()

    # Encode the query using the model
    encoded_query =  get_embeddings([query]).detach().numpy()
    # Searching for neighbors
    scores, samples = embeddings_dataset.get_nearest_examples("embeddings", encoded_query, k=3)

    end = time.time()

    samples_df = pd.DataFrame.from_dict(samples)
    samples_df["scores"] = scores
    samples_df.sort_values("scores", ascending=True, inplace=True)
    print(samples_df.drop(columns = 'embeddings'))

    compare_models.append({'Model': model_['Name'],
                           'Model_size': model_['Size'],
                           'Searching time': (end - start),
                           'Query': query,
                           'Score_of_best_result': samples_df['scores'][0],
                           'Best_finded_text': samples_df['text'][0]
    })


Map:   0%|          | 0/4 [00:00<?, ? examples/s]

  0%|          | 0/1 [00:00<?, ?it/s]

                                                text      scores
0  Пользуясь интегральным исчислением, решить зад...    0.000000
1  Заряд q распределен с постоянной поверхностной...  162.717560
2  Дан двухатомный идеальный газ, молекулы которо...  177.525269
                                                text      scores
0  Материальная точка массой 0,01 кг совершает га...  413.625763
1  Дан двухатомный идеальный газ, молекулы которо...  453.758850
2  Заряд q распределен с постоянной поверхностной...  501.306427
                                                text      scores
0  Материальная точка массой 0,01 кг совершает га...  378.742584
1  Заряд q распределен с постоянной поверхностной...  443.250427
2  Дан двухатомный идеальный газ, молекулы которо...  472.492493


Map:   0%|          | 0/4 [00:00<?, ? examples/s]

  0%|          | 0/1 [00:00<?, ?it/s]

                                                text      scores
0  Пользуясь интегральным исчислением, решить зад...    0.000000
1  Заряд q распределен с постоянной поверхностной...  127.737091
2  Дан двухатомный идеальный газ, молекулы которо...  168.837296
                                                text      scores
0  Дан двухатомный идеальный газ, молекулы которо...  216.552780
1  Материальная точка массой 0,01 кг совершает га...  283.893463
2  Заряд q распределен с постоянной поверхностной...  320.849915
                                                text      scores
0  Пользуясь интегральным исчислением, решить зад...  381.066040
1  Материальная точка массой 0,01 кг совершает га...  384.340637
2  Заряд q распределен с постоянной поверхностной...  394.663757


Some weights of the model checkpoint at cointegrated/rubert-tiny2 were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Map:   0%|          | 0/4 [00:00<?, ? examples/s]

  0%|          | 0/1 [00:00<?, ?it/s]

                                                text     scores
0  Пользуясь интегральным исчислением, решить зад...   0.000000
1  Заряд q распределен с постоянной поверхностной...  65.766357
2  Материальная точка массой 0,01 кг совершает га...  81.058731
                                                text      scores
0  Дан двухатомный идеальный газ, молекулы которо...   97.756622
1  Материальная точка массой 0,01 кг совершает га...  110.198959
2  Заряд q распределен с постоянной поверхностной...  133.005920
                                                text      scores
0  Материальная точка массой 0,01 кг совершает га...  132.394257
1  Заряд q распределен с постоянной поверхностной...  139.112167
2  Дан двухатомный идеальный газ, молекулы которо...  141.505310


In [10]:
#Print comparison df
compare_models = pd.DataFrame(compare_models)
compare_models

,Model,Model_size,Searching time,Query,Score_of_best_result,Best_finded_text
0,Sber SBERT 2021,1.71 GB,1.886488,"Пользуясь интегральным исчислением, решить зад...",0.000000,"Пользуясь интегральным исчислением, решить зад..."
1,Sber SBERT 2021,1.71 GB,0.327593,идеальный газ,413.625763,"Материальная точка массой 0,01 кг совершает га..."
2,Sber SBERT 2021,1.71 GB,0.281435,маятник,378.742584,"Материальная точка массой 0,01 кг совершает га..."
3,DeepPavlov RUBERT 2021,711 MB,0.498390,"Пользуясь интегральным исчислением, решить зад...",0.000000,"Пользуясь интегральным исчислением, решить зад..."
4,DeepPavlov RUBERT 2021,711 MB,0.091840,идеальный газ,216.552780,"Дан двухатомный идеальный газ, молекулы которо..."
5,DeepPavlov RUBERT 2021,711 MB,0.089136,маятник,381.066040,"Пользуясь интегральным исчислением, решить зад..."
6,RUBERT tiny 2022,120 MB,0.034222,"Пользуясь интегральным исчислением, решить зад...",0.000000,"Пользуясь интегральным исчислением, решить зад..."
7,RUBERT tiny 2022,120 MB,0.012290,идеальный газ,97.756622,"Дан двухатомный идеальный газ, молекулы которо..."
8,RUBERT tiny 2022,120 MB,0.005648,маятник,132.394257,"Материальная точка массой 0,01 кг совершает га..."


## **Conclusions**
DeepPavlov's Rubert model is quick, small and shows better searching results.
UPD: RUBERT tiny is also good. Need more tests with real dataset

# **Final Model**

In [11]:
# Define tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased-sentence")
model = AutoModel.from_pretrained("DeepPavlov/rubert-base-cased-sentence")

In [12]:
# mapping for all dataset
embeddings_dataset = dataset.map(lambda x: {"embeddings": get_embeddings(x["text"]).detach().numpy()[0]})

Map:   0%|          | 0/4 [00:00<?, ? examples/s]

In [13]:
embeddings_dataset

Dataset({
    features: ['text', 'embeddings'],
    num_rows: 4
})

In [14]:
# save dataset
embeddings_dataset.save_to_disk('exercise')

# load dataset
#from datasets import load_from_disk
#embeddings_dataset = load_from_disk('exercise')
#embeddings_dataset

Saving the dataset (0/1 shards):   0%|          | 0/4 [00:00<?, ? examples/s]

##FAISS

FAISS uses an **algorithm to efficiently compute the distances** between vectors and organize them in a way that allows for **fast nearest neighbor search**. The algorithm uses a combination of quantization and indexing techniques to divide the vector space into smaller subspaces, which makes the search faster and more efficient.

In [15]:
# add faiss index to embeddings
embeddings_dataset.add_faiss_index(column="embeddings")

  0%|          | 0/1 [00:00<?, ?it/s]

Dataset({
    features: ['text', 'embeddings'],
    num_rows: 4
})

In [16]:
# save faiss index
embeddings_dataset.save_faiss_index('embeddings', 'my_index.faiss')

In [17]:
# load index

#ds.load_faiss_index('embeddings', 'my_index.faiss')

In [18]:
query = "центр масс полукруга"
query_embedding = get_embeddings([query]).detach().numpy()
query_embedding.shape

(1, 768)

In [19]:
scores, samples = embeddings_dataset.get_nearest_examples("embeddings", query_embedding, k=5)

In [20]:
print(samples['text'][0])

Пользуясь интегральным исчислением, решить задачи 7.12, 7.13 и 7.14 аналитически. 7.12. Пользуясь численными методами, найти положение центра масс полукруга. Для простоты расчета положить R==1,00. 7.13. Пользуясь численными методами, найти положение центра масс полушара. 7.14.Пользуясь численными методами, найти положение центра масс прямого кругового конуса высотой h=1, образующая которого составляет угол а с высотой.


In [21]:
samples_df = pd.DataFrame.from_dict(samples)
samples_df["scores"] = scores
samples_df.sort_values("scores", ascending=True, inplace=True)
samples_df

,text,embeddings,scores
0,"Пользуясь интегральным исчислением, решить зад...","[-0.5170160531997681, -1.1109189987182617, 0.2...",381.005463
1,"Материальная точка массой 0,01 кг совершает га...","[-0.9238566160202026, -1.3108799457550049, 0.9...",437.180267
2,Заряд q распределен с постоянной поверхностной...,"[-0.9064199924468994, -1.1442804336547852, 0.8...",453.841858
3,"Дан двухатомный идеальный газ, молекулы которо...","[-0.25475162267684937, -0.943519115447998, 0.6...",463.054199


The scores returned by the FAISS index represent the distances between the query embedding and the nearest embeddings in the dataset. **The lower the score, the closer the corresponding example** is to the query embedding.

## Testing

In [22]:
# Define a function for semantic search
def semantic_search(query, ds=embeddings_dataset, column_to_print='text', k=5):
    # Encode the query using the model
    encoded_query =  get_embeddings([query]).detach().numpy()

    # Search for the most similar documents
    scores, retrieved_examples = ds.get_nearest_examples("embeddings", encoded_query, k=5)
    return retrieved_examples[column_to_print][:k]

In [23]:
# Testing
%%time
semantic_search('ан двухатомный идеальный газ, молекулы которого обладают электрическим дипольным моментом', k=3)

CPU times: user 99.5 ms, sys: 705 µs, total: 100 ms
Wall time: 102 ms


['Дан двухатомный идеальный газ, молекулы которого обладают электрическим дипольным моментом d {|d| = d0 = const}. Газ находится в постоянном однородном электрическом поле e0 и имеет температуру Т. Требуется: а) найти добавок к свободной энергии рассматриваемого идеального газа, обусловленный взаимодействием дипольных моментов d с электрическим полем е0. Число молекул в газе N, объем газа V; б) зная результат пункта а), найти вклад в теплоемкость газа, обусловленный взаимодействием диполей d с полем e0; в) найти распределение по углам электрических дипольных моментов d в поле e0; г) зная результат пункта в), найти выражение для диэлектрической проницаемости газа в случае слабого поля e0.',
 'Материальная точка массой 0,01 кг совершает гармонические колебания с периодом 2 с. Полная энергия колеблющейся точки 10-4 Дж. Найти амплитуду колебаний, написать уравнение колебаний, найти наибольшее значение силы, действующей на точку.',
 'Заряд q распределен с постоянной поверхностной плотностью

# Next Word Prediction

In [24]:
class AutoCompleteModel(object):
  def __init__(self,
               unknown_token='<unk>',
               start_token='<s>',
               end_token='<e>',
               k=1):
    self._unknown_word = '<unk>'
    self._start_token = start_token
    self._end_token = end_token

    self._tokenized_sentences = None
    self._k = k # smoothing prameter

    self._word_frequency = Counter() # keys are the closed vocab

    self._ngram_word_frequency = defaultdict(lambda: 0)
    self._ngram_plus1_word_frequency = defaultdict(lambda: 0)

    self._no_match_threshold = 5

  def tokenize(self, sentences):
    # Tokenize the sentences
    nltk.download('punkt')
    self._tokenized_sentences = [nltk.word_tokenize(sentence.lower()) for sentence in tqdm(sentences, desc="Tokenize")]

  def train(self, minimum_freq=5, ngram=3):
    self._minimum_freq = minimum_freq
    self._ngram = ngram

    # Prepare word vocab through frequency counter
    self._calculate_word_frequency()
    self._vocab = list( self._word_frequency.keys()) + [self._unknown_word, "<e>"]

    self._filter_low_freq_words() # simulate unknown words

    # Normalize data i.e replace less frequent words with unknown tag
    self._tokenized_sentences = self._tokenize_n_normalize(self._tokenized_sentences)

    # Prepare ngram word frequency
    self._ngram_word_frequency = self._count_n_grams(self._tokenized_sentences, self._ngram)
    self._ngram_plus1_word_frequency = self._count_n_grams(self._tokenized_sentences, self._ngram+1)


  def _calculate_word_frequency(self):
    '''
    Counts word counts
    '''
    for tokenized_sentence in tqdm(self._tokenized_sentences, desc="Word Frequency"):
      self._word_frequency.update(tokenized_sentence)

  def _filter_low_freq_words(self):
    '''
    Filter words whose count are less than threshold
    '''
    words = self._word_frequency.keys()
    words_to_be_deleted = []

    for word in words:
      if self._word_frequency[word] < self._minimum_freq:
          words_to_be_deleted.append(word)

    for word in words_to_be_deleted:
      del self._word_frequency[word]

  def _tokenize_n_normalize(self, tokenized_sentences):
    '''
    Remove all words which not part of vocab and replace it with unknown tag
    '''
    new_sentences = []
    for sentence in tqdm(tokenized_sentences, desc="Normalize"):
      new_sentence = []
      for token in sentence:
        if self._word_frequency[token] != 0:
          new_sentence.append(token)
        else:
          new_sentence.append(self._unknown_word)
      new_sentences.append(new_sentence)
    return new_sentences

  def _count_n_grams(self, tokenized_sentences, ngram):
    '''
    Creates n-gram from tokenized sentence and counts the same
    '''
    freq = defaultdict(lambda: 0)
    for sentence in tqdm(tokenized_sentences, desc="NGrams"):
      sentence = [self._start_token] * ngram + sentence + [self._end_token]
      m = len(sentence) if ngram == 1 else len(sentence) - 1
      for i in range(m):
        ngram_token = sentence[i:i+ngram]
        #freq[tuple(ngram_token)] += 1
        # tuples can't be used as key in JSON
        freq[" ".join(ngram_token)] += 1
    return freq

  def _estimate_probability(self, word, previous_ngram):
    vocab_size = len(self._word_frequency)
    #previous_ngram = tuple(previous_ngram)
    if type(previous_ngram) != list:
      previous_ngram = [previous_ngram]
    previous_ngram = " ".join(previous_ngram)
    previous_ngram_count = self._ngram_word_frequency.get(previous_ngram, 0)
    if previous_ngram_count == 0:
      # print("Warning no match found for entered words!")
      return 0
    denominator = previous_ngram_count + self._k * len(self._vocab)
    n_plus1_gram = previous_ngram + " " + word
    n_plus1_gram_count =  self._ngram_plus1_word_frequency.get(n_plus1_gram, 0)
    numerator = n_plus1_gram_count + self._k
    probability = numerator / denominator
    return probability

  def _estimate_probabilities(self, previous_ngram):
    probabilities = {}
    # previous_n_gram = tuple(previous_n_gram)
    if type(previous_ngram) != list:
      previous_ngram = [previous_ngram]
    previous_ngram = " ".join(previous_ngram).lower()
    for word in self._vocab:
      probabilities[word] = self._estimate_probability(word, previous_ngram)
    return probabilities

  def suggestions(self, previous_tokens, num_suggestions=5, start_with=None):
    """
    previous_tokens: N-Gram tuple of previous tokens
    num_sugestions :
    start_with:
    """
    previous_ngram = previous_tokens[-self._ngram:]
    probabilities = self._estimate_probabilities(previous_ngram)
    probs = probabilities.items()
    probs = filter(lambda t: t[1]>0, probs)
    if start_with:
       probs = filter(lambda t: t[0].startswith(start_with), probs)
    probs = sorted(probs, key=lambda t: t[1], reverse=True)
    words = map(lambda t: t[0], probs)
    words = list(words)
    return words[:num_suggestions]


  def save_as_json(self, name):
    data = {}
    data["ngram_word_frequency"] = self._ngram_word_frequency #json.dumps(self._ngram_word_frequency, indent = 4)
    data["ngram_plus1_word_frequency"] = self._ngram_plus1_word_frequency #json.dumps(self._ngram_plus1_word_frequency, indent = 4)
    data["vocab"] = self._vocab
    data["ngram"] = self._ngram

    with open(name, "w", encoding='utf-8') as file:
      json.dump(data, file, ensure_ascii=False, indent=4)

  def load_from_json(self, file_path):
    data = json.load(open(file_path))
    self._ngram_word_frequency = data["ngram_word_frequency"]
    self._ngram_plus1_word_frequency =data["ngram_plus1_word_frequency"]
    self._vocab = data["vocab"]
    self._ngram = data["ngram"]


In [25]:
%%time
model = AutoCompleteModel()
model.tokenize(dataset['text'])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
Tokenize: 100%|██████████| 4/4 [00:00<00:00, 412.24it/s]

CPU times: user 289 ms, sys: 59.5 ms, total: 349 ms
Wall time: 516 ms


In [26]:
model.train(minimum_freq=0, ngram=2)
model.save_as_json("bigram-autocompleter.json")

NGrams: 100%|██████████| 4/4 [00:00<00:00, 9776.93it/s]


In [27]:
%%time
model.suggestions(["молекулы", "которого"])

CPU times: user 353 µs, sys: 108 µs, total: 461 µs
Wall time: 465 µs


['обладают', 'дан', 'двухатомный', 'идеальный', 'газ']

In [28]:
model.train(minimum_freq=0, ngram=3)
model.save_as_json("threegram-autocompleter.json")

NGrams: 100%|██████████| 4/4 [00:00<00:00, 2872.32it/s]
